In [106]:
import rethinkdb as r
import delorean
from delorean import stops, parse, Delorean, epoch
from datetime import timedelta
from retrying import retry
import time
import datetime
import sys
import os
import praw
import logging; logger=logging.getLogger(); logger.setLevel('INFO')

# Global helpers
c = rethinkdb.connect(os.getenv('RETHINKDB_HOST', 'rethinkdb.reddit-analyzer'), os.getenv('RETHINKDB_PORT', 28015))
UA = '/r/%s flair analyzer by /u/coffenbacher for https://charlesoffenbacher.com' % sys.argv[1]
pr = praw.Reddit(user_agent=UA, store_json_result='True')


In [113]:
# Seed start dates
r.db('reddit').table('progress').insert({'id': 'current_cfb', 'epoch':1199181202}, conflict='replace').run(c)



{u'deleted': 0,
 u'errors': 0,
 u'inserted': 0,
 u'replaced': 1,
 u'skipped': 0,
 u'unchanged': 0}

In [21]:
# Scratchpad

query = 'timestamp:1427232122..1437773560'
submissions = list(r.search(query, subreddit='cfb', sort='new', limit=10, syntax='cloudsearch'))

INFO:requests.packages.urllib3.connectionpool:Starting new HTTPS connection (1): api.reddit.com


In [112]:
from bigquery import get_client

client = get_client('stable-balancer-95721', 
                    service_account='account-1@stable-balancer-95721.iam.gserviceaccount.com',
                    private_key_file='../redditbigquerykey.p12', 
                    readonly=False)

submission_schema = [
    {'mode': 'nullable', 'name': 'downs', 'type': 'integer'},
     {'mode': 'nullable', 'name': 'subreddit', 'type': 'string'},
     {'mode': 'nullable', 'name': 'selftext', 'type': 'string'},
     {'mode': 'nullable', 'name': 'link_flair_text', 'type': 'string'},
     {'mode': 'nullable', 'name': 'id', 'type': 'string'},
     {'mode': 'nullable', 'name': 'gilded', 'type': 'integer'},
     {'mode': 'nullable', 'name': 'name', 'type': 'string'},
     {'mode': 'nullable', 'name': 'author', 'type': 'string'},
     {'mode': 'nullable', 'name': 'num_comments', 'type': 'integer'},
     {'mode': 'nullable', 'name': 'title', 'type': 'string'},
     {'mode': 'nullable', 'name': 'created_utc', 'type': 'timestamp'},
     {'mode': 'nullable', 'name': 'score', 'type': 'integer'},
     {'mode': 'nullable', 'name': 'author_flair_text', 'type': 'string'},
     {'mode': 'nullable', 'name': 'ups', 'type': 'integer'}
]

comment_schema = [
    {'mode': 'nullable', 'name': 'body', 'type': 'string'},
 {'mode': 'nullable', 'name': 'downs', 'type': 'integer'},
 {'mode': 'nullable', 'name': 'link_id', 'type': 'string'},
 {'mode': 'nullable', 'name': 'subreddit', 'type': 'string'},
 {'mode': 'nullable', 'name': 'link_flair_text', 'type': 'string'},
 {'mode': 'nullable', 'name': 'id', 'type': 'string'},
 {'mode': 'nullable', 'name': 'gilded', 'type': 'integer'},
 {'mode': 'nullable', 'name': 'name', 'type': 'string'},
 {'mode': 'nullable', 'name': 'author', 'type': 'string'},
 {'mode': 'nullable', 'name': 'author_flair_text', 'type': 'string'},
 {'mode': 'nullable', 'name': 'created_utc', 'type': 'timestamp'},
 {'mode': 'nullable', 'name': 'score', 'type': 'integer'},
 {'mode': 'nullable', 'name': 'ups', 'type': 'integer'}
]

def create_tables(subreddit):
    client.create_table('reddit', '%s_submissions' % subreddit, submission_schema)
    client.create_table('reddit', '%s_comments' % subreddit, comment_schema)

create_tables('cfb')

INFO:googleapiclient.discovery:URL being requested: POST https://www.googleapis.com/bigquery/v2/projects/stable-balancer-95721/datasets/reddit/tables?alt=json
INFO:oauth2client.client:Attempting refresh to obtain initial access_token
INFO:oauth2client.client:Refreshing access_token
INFO:googleapiclient.discovery:URL being requested: POST https://www.googleapis.com/bigquery/v2/projects/stable-balancer-95721/datasets/reddit/tables?alt=json


In [63]:
# {
#     'id': s.id,
#     'author': s.author.name,
#     'created_utc': s.created_utc,
#     'link_flair_text': s.link_flair_text,
#     'author_flair_text': s.author_flair_text,
#     'num_comments': s.num_comments,
#     'score': s.score,
#     'ups': s.ups,
#     'downs': s.downs,
#     'selftext': s.selftext,
#     'title': s.title,
#     'gilded': s.gilded,
#     'subreddit': s.subreddit.name,
#     'name': s.name
# }

[{'mode': 'nullable', 'name': 'downs', 'type': 'integer'},
 {'mode': 'nullable', 'name': 'subreddit', 'type': 'string'},
 {'mode': 'nullable', 'name': 'selftext', 'type': 'string'},
 {'mode': 'nullable', 'name': 'link_flair_text', 'type': 'string'},
 {'mode': 'nullable', 'name': 'id', 'type': 'string'},
 {'mode': 'nullable', 'name': 'gilded', 'type': 'integer'},
 {'mode': 'nullable', 'name': 'name', 'type': 'string'},
 {'mode': 'nullable', 'name': 'author', 'type': 'string'},
 {'mode': 'nullable', 'name': 'num_comments', 'type': 'integer'},
 {'mode': 'nullable', 'name': 'title', 'type': 'string'},
 {'mode': 'nullable', 'name': 'created_utc', 'type': 'float'},
 {'mode': 'nullable', 'name': 'score', 'type': 'integer'},
 {'mode': 'nullable', 'name': 'author_flair_text', 'type': 'string'},
 {'mode': 'nullable', 'name': 'ups', 'type': 'integer'}]

In [83]:
# {
#     'id': c.id,
#     'author': c.author.name,
#     'created_utc': c.created_utc,
#     'link_id': c.link_id,
#     'link_flair_text': c.submission.link_flair_text,
#     'author_flair_text': c.author_flair_text,
#     'score': c.score,
#     'ups': c.ups,
#     'downs': c.downs,
#     'body': c.body,
#     'gilded': c.gilded,
#     'subreddit': c.subreddit.name,
#     'name': c.name
# }


[{'mode': 'nullable', 'name': 'body', 'type': 'string'},
 {'mode': 'nullable', 'name': 'downs', 'type': 'integer'},
 {'mode': 'nullable', 'name': 'link_id', 'type': 'string'},
 {'mode': 'nullable', 'name': 'subreddit', 'type': 'string'},
 {'mode': 'nullable', 'name': 'link_flair_text', 'type': 'string'},
 {'mode': 'nullable', 'name': 'id', 'type': 'string'},
 {'mode': 'nullable', 'name': 'gilded', 'type': 'integer'},
 {'mode': 'nullable', 'name': 'name', 'type': 'string'},
 {'mode': 'nullable', 'name': 'author', 'type': 'string'},
 {'mode': 'nullable', 'name': 'author_flair_text', 'type': 'string'},
 {'mode': 'nullable', 'name': 'created_utc', 'type': 'float'},
 {'mode': 'nullable', 'name': 'score', 'type': 'integer'},
 {'mode': 'nullable', 'name': 'ups', 'type': 'integer'}]

1437743134.0